# PROYECTO 3 
________________________________________________________________________________________________
## ANALISIS DE DATOS DE AIRBNB EN LA CIUDAD DE MADRID

Con el objetivo de analizar el comportamiento de los alojamientos de Airbnb en una ciudad concreta, se nos
ha encargado el desarrollo de un proyecto de análisis de datos que permita identificar patrones, tendencias
y oportunidades de mejora. Este análisis ayudará a comprender mejor cómo se distribuyen los alojamientos,
qué factores influyen en el precio, qué características están asociadas a mejores valoraciones y cómo se
comporta la oferta en distintos barrios o zonas de la ciudad.

### FASE 1: Análisis Exploratorio de Datos(EDA).

_____________________________________________

En esta fase se realiza un anáisis exploratorio detallado para comprender el conjunto de datos y sus carcaterísticas.

Objetivo: comprender el contexto general de los datos y detectar posibles problemas de calidad.

In [ ]:
# Cargando librerías

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Cargando el dataset
df_listings = pd.read_csv('listings.csv')
df_reviews = pd.read_csv('reviews.csv')


In [ ]:
df_listings = pd.read_csv('listings.csv')

In [ ]:
df_listings.head()

In [ ]:
df_listings.columns

In [ ]:
#EDA 

def eda(df, name):
    
    print(f"\n{'='*40}")
    print(f"EDA DEL DATAFRAME: {name.upper()}")
    print(f"{'='*40}\n")
    
    print("========== RESUMEN GENERAL ==========")
    print(f"Filas x Columnas (shape): {df.shape}")
    print("\nColumnas:")
    print(df.columns.tolist())

    print("\nDtypes:")
    print(df.dtypes)

    print("\nNulos por columna:")
    print(df.isnull().sum())

    print("\n========== DESCRIBE NUMÉRICO ==========")
    print(df.describe().T)

    print("\n========== DESCRIBE CATEGÓRICO (object/category) ==========")
    print(df.describe(include=["O"]).T)

    print("\n========== HEAD ==========")
    display(df.head())
    print("\n========== TAIL ==========")
    display(df.tail())
    print("\n========== SAMPLE ==========")
    display(df.sample())
    print("\n========== VALUE COUNTS (por columna categórica) ==========")
    col_categoricas =  df.select_dtypes(include=["object", "category"]).columns.tolist()

    for c in col_categoricas:
            print(f"\n--- {c} ---")
            print(df[c].value_counts)

    print("\n========== DUPLICADOS ==========")

    print(f"Duplicados: {df.duplicated().sum()}")

    print("\n========== HISTOGRAMAS NUMÉRICOS ==========")

    df.hist(bins=20, figsize=(25,25))
    plt.show()
    


In [ ]:
def limpieza (df, name):
    
    print(f"limpieza del dataframe: {name}")
    
    if df.duplicated().sum() > 0:
        df = df.drop_duplicates()
    
    df = df.drop(['name','license','host_name'], axis=1)
    
    return df

In [ ]:
df.sample()

# Conclusiones del EDA – `df_listings`

## 1. ¿Cuántos alojamientos únicos contiene el dataset?
El dataset contiene **25.000 alojamientos únicos**, identificados mediante la columna `id`.  
No se han detectado registros duplicados, lo que indica una **buena calidad estructural** de los datos en este aspecto.

---

## 2. ¿Qué tipos de alojamiento existen y cuál es el más común?
Existen **cuatro tipos de alojamiento**, siendo claramente dominante el tipo **“Entire home/apt”**, que concentra la mayor parte de los anuncios.  
Esto sugiere que la oferta está principalmente orientada a **alquileres completos**, frente a habitaciones privadas o compartidas.

---

## 3. ¿Cómo se distribuyen los alojamientos por barrio o zona?
La distribución de los alojamientos no es homogénea. Se observa una **fuerte concentración en zonas céntricas**, destacando el distrito **Centro** como principal área de oferta.  
A nivel de barrio, **Embajadores** es el más representado, seguido de otros barrios céntricos y bien conectados.

---

## 4. ¿Existen barrios con una concentración especialmente alta de alojamientos?
Sí. El distrito **Centro** y algunos de sus barrios presentan una **concentración significativamente superior** al resto de la ciudad.  
Este patrón sugiere un **desequilibrio espacial de la oferta**, probablemente vinculado a factores turísticos, de accesibilidad y demanda.

---

## 5. ¿Qué variables presentan valores nulos y en qué proporción?
Se identifican valores nulos relevantes en varias columnas:

- **`price`**: aproximadamente **24 %** de valores nulos.  
- **`last_review`** y **`reviews_per_month`**: alrededor de **20 %**, asociados a alojamientos sin reseñas o sin actividad reciente.  
- **`license`**: más del **60 %** de valores nulos, lo que limita su utilidad analítica.  
- **`host_name`**: valores nulos residuales y poco significativos.

Estos nulos deberán tratarse de forma diferenciada según el objetivo del análisis.

---

## 6. ¿Hay columnas cuyo contenido no aporta valor al análisis?
Sí. Algunas columnas presentan **bajo valor analítico** en esta fase exploratoria:

- **`name`**: texto libre con alta cardinalidad y escaso valor agregado.  
- **`license`**: elevado porcentaje de nulos y gran heterogeneidad de formatos.  
- **`host_name`**: información nominal sin impacto directo en el análisis exploratorio.

Estas variables podrían **eliminarse o reservarse** para análisis específicos.

---

## 7. ¿Existen valores atípicos en el precio de los alojamientos? ¿Cómo afectan a la media?
Sí, el precio presenta **valores atípicos muy elevados**, con máximos muy alejados del rango intercuartílico.  
Como consecuencia, la **media del precio se ve inflada** y no representa adecuadamente el comportamiento típico del mercado.  
La **mediana** se presenta como una medida **más robusta y representativa**.

---

## 8. Evaluación general de la calidad de los datos
En conjunto, el dataset presenta:

- Buena calidad estructural (sin duplicados y con tipos de datos coherentes).  
- Problemas habituales en datos reales: **valores nulos relevantes, outliers y variables poco informativas**.  
- Información suficiente para avanzar hacia análisis exploratorios más profundos, siempre que se apliquen **estrategias adecuadas de limpieza y tratamiento de datos**.

El objetivo de esta fase de EDA —comprender el contexto general de los datos y detectar posibles problemas de calidad— se considera **satisfactoriamente alcanzado**.


### ANALISIS

In [ ]:
def analisis(df, name):
    
    print(f"\n{'='*40}")
    print(f"ANÁLISIS DEL DATAFRAME: {name.upper()}")
    print(f"{'='*40}\n")
    """
    El dataset df_listings contiene 25.000 alojamientos únicos sin registros duplicados, con una oferta
    mayoritariamente concentrada en viviendas completas y en zonas céntricas. Presenta valores nulos
    relevantes en variables clave como el precio y la licencia, así como valores atípicos elevados que
    afectan a la media del precio, por lo que requiere un tratamiento previo antes de realizar análisis
    más profundos, aunque su estructura general es adecuada para continuar con el proceso de EDA.
    """
    print("========== Nº DE ALOJAMIENTOS ÚNICOS ==========")
    # ¿Cuántos alojamientos únicos hay en el dataset?
    
    print(f"\nEl dataset incluye {df['id'].nunique()} alojamientos únicos, de un total de {df.shape[0]} filas totales.\n")
    
    print("========== TIPOS DE ALOJAMIENTO ==========")
    # ¿Qué tipos de alojamientos hay y cuál es el más común?
    
    print(f"\nExisten {df['room_type'].nunique()} tipos de alojamientos únicos:{(df['room_type'].unique())}. \nEl más común es {df['room_type'].value_counts().index[0]} con {df['room_type'].value_counts().iloc[0]} alojamientos.\n")
    
    print("========== DISTRIBUCION ALOJAMIENTOS ==========")
    # ¿Cómo se distribuye los alojamientos por barrio y por zona?
    
    print('\nObservamos que los alojamientos se distribuyen por barrio y por zona:')
    
    alojamientos_por_barrio = df.groupby([df['neighbourhood_group'].str.upper(), 'neighbourhood'])['id'].count().sort_values(ascending=False)
    
    print(alojamientos_por_barrio) 
    
    print('\nLos alojamientos no se distribuyen de forma homogénea por la ciudad: se concentran en distritos específicos, \nmás céntricos o mejor conectados, mientras que zonas periféricas presentan una oferta mucho más limitada. \nEsto sugiere patrones de localización ligados a accesibilidad, atractivo turístico y servicios disponibles.\n')    
    
    alojamientos_por_distrito = df.groupby(['neighbourhood_group'])['id'].count().sort_values(ascending=False)  
    alojamientos_por_distrito.plot(kind='bar', figsize=(15,5))
    plt.title('Número de alojamientos por distrito')
    plt.xlabel('Distrito')
    plt.ylabel('Número de alojamientos')
    plt.xticks(rotation=90)
    plt.show() 
    
    print("========== ZONAS CON MAYOR CONCENTRACION ==========")
    # ¿Existen barrios con una concentración especialmente alta de alojamientos?
    
    df.groupby(['neighbourhood', 'neighbourhood_group'])['id'].count().sort_values(ascending=False).head(10)
    
    print(f"\nExisten barrios con una concentración especialmente alta de alojamientos: \n{df['neighbourhood'].value_counts().head(10)}")
    
    print('\nLos barrios con mayor concentración de alojamientos son aquellos que probablemente sean más turísticos o céntricos, \nlo que puede indicar una mayor demanda y atractivo para los visitantes. \nEsto también puede reflejar la presencia de atracciones turísticas, buena conectividad y servicios disponibles en esos barrios, \nlo que los hace más atractivos para los anfitriones y los huéspedes.\n')
    
    print("========== NULOS ==========")
    #¿Qué variables presentan valores nulos y en qué proporción?
    
    print('\nPorcentaje de nulos por columna:\n')
    
    print(f"{(df.isnull().sum() / df.shape[0] * 100).round(2)}%\n")
    
    print("========== VARIABLES DE BAJO VALLOR ANALÍTICO ==========")
    #¿Hay columnas cuyo contenido no aporta valor al análisis y podrían eliminarse?
    
    print(f"\nComprobando variables con un solo valor único:\n{df.columns[df.nunique() == 1].tolist()} No existen.")
    print(f"\nSin embargo, algunas columnas presentan bajo valor analítico en esta fase exploratoria:")
    print('\nname: texto libre con alta cardinalidad y escaso valor agregado.' ) 
    print('\nlicense: elevado porcentaje de nulos y gran heterogeneidad de formatos.')  
    print('\nhost_name: información nominal sin impacto directo en el análisis exploratorio.')
    print('\nEstas variables podrían eliminarse para este análisis.')
    
    print('\n')
    
    print("========== OUTLIERS ==========")
    #¿Existen valores atípicos en el precio de los alojamientos? ¿Cómo afectan a la media?
    
    print(f"\nExisten valores atípicos en el precio de los alojamientos. \nEl precio medio es {df['price'].mean():.2f}, pero la mediana es {df['price'].median():.2f}, \nlo que indica que los valores atípicos están influyendo en la media y elevándola por encima de la mediana.") 
    print('\nLos valores atípicos en el precio pueden distorsionar la media, haciéndola menos representativa del precio típico de los alojamientos. \nLa mediana, al ser menos sensible a los valores extremos, proporciona una mejor medida de tendencia central en este caso, \nindicando que la mayoría de los alojamientos tienen precios más bajos que la media sugiere debido a la presencia de algunos alojamientos muy caros.')

    print('\n\nEl objetivo de esta fase -entender el contexto general de los datos y detectar posibles problemas de calidad- se considera alcanzado.')
    
    return df

df = analisis(df , 'LISTINGS')

In [ ]:
df_listings = df_listings.drop(['name','license','host_name'], axis=1)

In [ ]:
df_listings.sample()

In [ ]:
df.to_csv('listings_eda.csv')


In [ ]:
df.groupby([df['neighbourhood_group'].str.upper(), 'neighbourhood'])['id'].count().sort_values(ascending=False).head(10)

In [ ]:
df.groupby(['neighbourhood', 'neighbourhood_group'])['id'].count().sort_values(ascending=False)

alojamientos_por_barrio = df.groupby(['neighbourhood_group', 'neighbourhood'])['id'].count().sort_values(ascending=False)
print(alojamientos_por_barrio)    
alojamientos_por_distrito = df.groupby(['neighbourhood_group'])['id'].count().sort_values(ascending=False)  
alojamientos_por_distrito.plot(kind='bar', figsize=(15,5))
plt.title('Número de alojamientos por distrito')
plt.xlabel('Distrito')
plt.ylabel('Número de alojamientos')
plt.xticks(rotation=90)
plt.show()

In [ ]:
analisis(df)